In [1]:
import os
import json
import requests as re
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import scipy
import math
import random
from sklearn.metrics import roc_auc_score, mean_absolute_error,mean_squared_error

from sklearn import preprocessing
import sklearn
import unidecode
import seaborn as sns
from  concurrent.futures import ThreadPoolExecutor, as_completed
# from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

pd.options.display.max_rows = 339
pd.options.display.max_columns = 339

pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

from catboost import CatBoostRegressor

### Preprocessing

In [3]:
df_ = pd.read_pickle('../vivino_base/datasets/df_main_v4.pkl')

In [4]:
df_.head()

,wine_id,wine_name,wine_type_id,region,region_id,country,winery,winery_id,style_name,style_body,style_body_description,acidity,acidity_description,grape_1,grape_2,grape_3,grape_4,grape_5,ratings_count_wine,ratings_average_wine,volume_ml,non_vintage,alcohol,is_natural,img_perfect_url,wine_critic_reviews,awards,img_default_url,ratings_count_vintages_list,ratings_average_vintages_list,year_list,price_avg
0,4235931,Cabernet Sauvignon,1,Rutherford,105.00,United States,Xanthos,72210.00,Napa Valley Cabernet Sauvignon,5.00,Very full-bodied,2.00,Medium,Cabernet Sauvignon,None,None,None,None,890.00,4.00,NaN,None,14.50,None,None,None,None,//images.vivino.com/labels/9WkRHpnJSPCLkusYIuA...,[890.0],[4.0],[0],18.35
1,8105703,Pinot Noir,1,Monterey,1893.00,United States,Sovino,277519.00,Californian Pinot Noir,3.00,Medium-bodied,3.00,High,Pinot Noir,None,None,None,None,5.00,NaN,NaN,None,14.50,None,None,None,None,//images.vivino.com/labels/cowrCbohRI2fa3oYY0x...,None,None,None,NaN
2,4792703,Nero d'Avola,1,Sicilia,3334.00,Italy,Jonico,124810.00,Southern Italy Red,4.00,Full-bodied,3.00,High,Primitivo,Aglianico,Negroamaro,Nero d'Avola,None,209.00,3.60,NaN,None,NaN,None,None,None,None,//images.vivino.com/labels/bJtFXouqS26wHs38o7N...,[209.0],[3.6],[0],23.00
3,4977812,Kerner Feinherb,2,Mosel,545.00,Germany,Bremer Ratskeller,36057.00,None,NaN,None,NaN,None,None,None,None,None,None,14.00,NaN,NaN,None,NaN,None,None,None,None,//images.vivino.com/labels/F2EouX_2Q-GgNEvSW5t...,None,None,None,NaN
4,5011731,Nero d'Avola,1,Terre Siciliane,579.00,Italy,Kalsa,50432.00,Southern Italy Red,4.00,Full-bodied,3.00,High,Primitivo,Aglianico,Negroamaro,Nero d'Avola,None,68.00,3.50,NaN,None,NaN,None,None,None,None,//images.vivino.com/labels/vVIPx5fFT2e_v4CTPaU...,[68.0],[3.5],[0],26.00


In [5]:
df = df_.copy()

In [6]:
df = df.dropna(subset = ['price_avg','ratings_count_wine','ratings_average_wine']).drop([
    'volume_ml','non_vintage',],axis = 1)

In [7]:
df_r_ = pd.read_pickle('comments/reviews_v0.pkl')

In [8]:
df_r_.head()

,user_id,vintage_id,wine_id,rating,date
0,14629027,10890662,2434978,4.50,2017-06-01T09:42:47.000Z
1,48902778,10890662,2434978,5.00,2021-09-24T07:30:30.000Z
2,44683421,10890662,2434978,5.00,2021-01-24T07:37:30.000Z
3,3946442,8772403,2434978,4.50,2017-03-02T12:35:49.000Z
4,30482903,157552929,2434978,4.50,2019-08-11T08:01:07.000Z


In [9]:
df_r = df_r_[['wine_id', 'rating']].copy()

In [10]:
df_r = df_r[df_r['rating'].isin([i/2 for i in range(1, 11)])]

In [11]:
df_r_h = pd.DataFrame(df_r.groupby('wine_id')).copy()

In [12]:
df_r_h[1] = df_r_h[1].apply(lambda x: list(x['rating']))

In [13]:
df_r_h.columns = ['wine_id', 'ratings']

In [14]:
df_r = df_r_h.copy()

In [15]:
df_r

,wine_id,ratings
0,10,"[3.5, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, ..."
1,17,"[4.0, 3.0, 3.0, 1.0, 4.0, 4.0, 4.0, 4.0, 4.0, ..."
2,19,"[5.0, 4.5, 4.5, 4.5, 4.5, 4.5, 4.5, 5.0, 4.5, ..."
3,20,"[4.0, 4.0, 5.0, 4.0, 3.0, 3.5, 5.0, 4.0, 5.0, ..."
4,23,"[3.5, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 4.0, ..."
...,...,...
683599,11080802,[3.5]
683600,11080861,[3.5]
683601,11081335,[3.5]
683602,11081435,[4.5]


In [16]:
df = df.merge(df_r, how='inner', on='wine_id')

In [17]:
df['ratings_count'] = df['ratings'].apply(lambda x: len(x))

In [18]:
df.columns

Index(['wine_id', 'wine_name', 'wine_type_id', 'region', 'region_id',
       'country', 'winery', 'winery_id', 'style_name', 'style_body',
       'style_body_description', 'acidity', 'acidity_description', 'grape_1',
       'grape_2', 'grape_3', 'grape_4', 'grape_5', 'ratings_count_wine',
       'ratings_average_wine', 'alcohol', 'is_natural', 'img_perfect_url',
       'wine_critic_reviews', 'awards', 'img_default_url',
       'ratings_count_vintages_list', 'ratings_average_vintages_list',
       'year_list', 'price_avg', 'ratings', 'ratings_count'],
      dtype='object')

In [19]:
ml_col = df.columns[[0, 2, 4, 5, 7, 8, 9, 11, 13, 14, 15, 16, 17, 20, 21, 29, 30, 31]]

In [20]:
df_ml = df[ml_col]

In [21]:
df_ml.head()

,wine_id,wine_type_id,region_id,country,winery_id,style_name,style_body,acidity,grape_1,grape_2,grape_3,grape_4,grape_5,alcohol,is_natural,price_avg,ratings,ratings_count
0,4235931,1,105.00,United States,72210.00,Napa Valley Cabernet Sauvignon,5.00,2.00,Cabernet Sauvignon,None,None,None,None,14.50,None,18.35,"[4.0, 3.5, 4.0, 4.0, 3.5, 4.5, 3.0, 5.0, 5.0, ...",315
1,4792703,1,3334.00,Italy,124810.00,Southern Italy Red,4.00,3.00,Primitivo,Aglianico,Negroamaro,Nero d'Avola,None,NaN,None,23.00,"[4.0, 2.5, 3.0, 4.5, 4.0, 4.0, 4.0, 3.5, 3.5, ...",53
2,5011731,1,579.00,Italy,50432.00,Southern Italy Red,4.00,3.00,Primitivo,Aglianico,Negroamaro,Nero d'Avola,None,NaN,None,26.00,"[4.0, 3.0, 2.0, 3.0, 3.5, 3.0, 3.0, 3.5, 3.0, ...",21
3,7759560,2,65.00,United States,20388.00,Californian Sauvignon Blanc,3.00,3.00,Sauvignon Blanc,None,None,None,None,NaN,None,16.32,"[3.0, 2.0, 3.0, 3.5, 1.0, 4.0, 4.0]",7
4,2890794,1,4263.00,Argentina,206926.00,Argentinian Cabernet Sauvignon - Malbec,4.00,3.00,Cabernet Sauvignon,Malbec,None,None,None,NaN,None,145.14,"[5.0, 5.0, 5.0, 5.0, 4.5, 5.0, 5.0, 4.0, 5.0, ...",128


In [22]:
df_train = df_ml[df_ml.ratings_count > 880].reset_index().copy()

In [23]:
df_train.head()

,index,wine_id,wine_type_id,region_id,country,winery_id,style_name,style_body,acidity,grape_1,grape_2,grape_3,grape_4,grape_5,alcohol,is_natural,price_avg,ratings,ratings_count
0,76,69614,2,440.00,France,10544.00,Bordeaux White,3.00,3.00,Sauvignon Blanc,Sémillon,Muscadelle,None,None,NaN,None,8.88,"[4.0, 5.0, 4.0, 5.0, 4.0, 4.0, 4.0, 4.0, 4.0, ...",934
1,132,1099243,1,555.00,France,1668.00,Bordeaux Pomerol,4.00,2.00,Cabernet Sauvignon,Cabernet Franc,Malbec,Merlot,Petit Verdot,NaN,None,81.66,"[4.5, 4.5, 4.5, 4.5, 4.0, 4.5, 4.5, 4.0, 4.5, ...",1379
2,148,87676,1,673.00,Italy,244651.00,Italian Bolgheri,4.00,3.00,Cabernet Sauvignon,Merlot,Sangiovese,None,None,14.00,None,42.58,"[4.0, 3.5, 3.0, 3.5, 3.5, 3.5, 4.0, 4.0, 4.0, ...",2590
3,188,15142,1,956.00,Italy,2381.00,Northern Italy Red,4.00,3.00,Nebbiolo,Barbera,Dolcetto,None,None,15.00,None,14.95,"[4.0, 4.0, 4.0, 4.0, 4.0, 3.5, 4.0, 4.0, 4.0, ...",1318
4,313,768736,1,385.00,France,4905.00,Burgundy Red,2.00,3.00,Pinot Noir,None,None,None,None,NaN,None,14.35,"[4.0, 4.0, 4.0, 4.0, 3.5, 4.0, 3.5, 3.5, 4.0, ...",1901


In [24]:
df_for_prediction = df_ml[df_ml.ratings_count <= 880].reset_index().copy()

In [25]:
df_for_prediction.head()

,index,wine_id,wine_type_id,region_id,country,winery_id,style_name,style_body,acidity,grape_1,grape_2,grape_3,grape_4,grape_5,alcohol,is_natural,price_avg,ratings,ratings_count
0,0,4235931,1,105.00,United States,72210.00,Napa Valley Cabernet Sauvignon,5.00,2.00,Cabernet Sauvignon,None,None,None,None,14.50,None,18.35,"[4.0, 3.5, 4.0, 4.0, 3.5, 4.5, 3.0, 5.0, 5.0, ...",315
1,1,4792703,1,3334.00,Italy,124810.00,Southern Italy Red,4.00,3.00,Primitivo,Aglianico,Negroamaro,Nero d'Avola,None,NaN,None,23.00,"[4.0, 2.5, 3.0, 4.5, 4.0, 4.0, 4.0, 3.5, 3.5, ...",53
2,2,5011731,1,579.00,Italy,50432.00,Southern Italy Red,4.00,3.00,Primitivo,Aglianico,Negroamaro,Nero d'Avola,None,NaN,None,26.00,"[4.0, 3.0, 2.0, 3.0, 3.5, 3.0, 3.0, 3.5, 3.0, ...",21
3,3,7759560,2,65.00,United States,20388.00,Californian Sauvignon Blanc,3.00,3.00,Sauvignon Blanc,None,None,None,None,NaN,None,16.32,"[3.0, 2.0, 3.0, 3.5, 1.0, 4.0, 4.0]",7
4,4,2890794,1,4263.00,Argentina,206926.00,Argentinian Cabernet Sauvignon - Malbec,4.00,3.00,Cabernet Sauvignon,Malbec,None,None,None,NaN,None,145.14,"[5.0, 5.0, 5.0, 5.0, 4.5, 5.0, 5.0, 4.0, 5.0, ...",128


In [26]:
df_train['target_rating'] = df_train['ratings'].apply(lambda x: sum(x)/len(x))

In [27]:
import random

In [28]:
def rand_vals(seed, arr):
    res = []
    random.seed(seed)
    for i in range(len(arr)-1):
        v = arr[i] + (arr[i+1]-arr[i])*random.random()
        res.append(int(v))
    return res

In [29]:
def make_sample(df):
    df_res = pd.DataFrame(columns=list(df.columns)+['currency_rating', 'min_rating', 'max_rating', 'seed'])
    seed = 0
    for i in tqdm(df.index):
        ser_i = df.loc[i].copy()
        ratings = ser_i['ratings']
        seed += 1
        random.seed(seed)
        np.random.seed(seed)
        intervals = np.logspace(np.log2(30), np.log2(880), num=round(4+random.random()), base=2)
        vals = rand_vals(seed, intervals)
        for v in vals:
            ser_i['ratings'] = ratings_i = np.random.choice(ratings, size=v, replace=False)
            ser_i['ratings_count'] = ratings_count_i = ratings_i.shape[0]
            ser_i['currency_rating'] = currency_rating = np.mean(ratings_i)
            #sqrt(n) * eps / std = 1.96, std = 0.75
            eps = round(1.47 / (ratings_count_i**0.5), 4)
            ser_i['min_rating'] = currency_rating - eps
            ser_i['max_rating'] = currency_rating + eps
            ser_i['seed'] = seed
            df_res = pd.concat([df_res, ser_i.to_frame().T.copy()], ignore_index=True)
    return df_res

In [30]:
df_train_sample = make_sample(df_train)

100%|██████████| 5608/5608 [00:54<00:00, 103.06it/s]


In [31]:
df_train_sample

,index,wine_id,wine_type_id,region_id,country,winery_id,style_name,style_body,acidity,grape_1,grape_2,grape_3,grape_4,grape_5,alcohol,is_natural,price_avg,ratings,ratings_count,target_rating,currency_rating,min_rating,max_rating,seed
0,76,69614,2,440.00,France,10544.00,Bordeaux White,3.00,3.00,Sauvignon Blanc,Sémillon,Muscadelle,None,None,NaN,None,8.88,"[4.0, 4.0, 3.0, 4.5, 3.5, 5.0, 5.0, 4.0, 5.0, ...",38,4.05,4.00,3.76,4.24,1
1,76,69614,2,440.00,France,10544.00,Bordeaux White,3.00,3.00,Sauvignon Blanc,Sémillon,Muscadelle,None,None,NaN,None,8.88,"[3.0, 4.0, 4.0, 4.0, 4.0, 5.0, 4.0, 4.0, 5.0, ...",255,4.05,4.09,4.00,4.18,1
2,76,69614,2,440.00,France,10544.00,Bordeaux White,3.00,3.00,Sauvignon Blanc,Sémillon,Muscadelle,None,None,NaN,None,8.88,"[4.5, 4.0, 4.0, 5.0, 5.0, 3.5, 3.5, 4.5, 4.5, ...",739,4.05,4.05,4.00,4.11,1
3,132,1099243,1,555.00,France,1668.00,Bordeaux Pomerol,4.00,2.00,Cabernet Sauvignon,Cabernet Franc,Malbec,Merlot,Petit Verdot,NaN,None,81.66,"[4.0, 3.0, 4.0, 4.0, 5.0, 3.0, 5.0, 4.0, 3.5, ...",68,4.24,4.22,4.04,4.40,2
4,132,1099243,1,555.00,France,1668.00,Bordeaux Pomerol,4.00,2.00,Cabernet Sauvignon,Cabernet Franc,Malbec,Merlot,Petit Verdot,NaN,None,81.66,"[4.0, 5.0, 5.0, 5.0, 3.0, 3.0, 5.0, 4.5, 4.0, ...",157,4.24,4.27,4.15,4.39,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19601,285954,6433137,2,431.00,South Africa,15164.00,None,NaN,NaN,None,None,None,None,None,NaN,None,9.87,"[4.5, 3.0, 4.0, 5.0, 4.0, 4.0, 5.0, 5.0, 5.0, ...",808,4.18,4.16,4.11,4.22,5607
19602,286228,6763584,1,327.00,United States,255469.00,Washington State Cabernet Sauvignon,4.00,3.00,Cabernet Sauvignon,None,None,None,None,NaN,None,23.07,"[3.5, 4.0, 4.0, 4.5, 4.0, 4.0, 4.5, 5.0, 5.0, ...",60,4.06,4.02,3.83,4.21,5608
19603,286228,6763584,1,327.00,United States,255469.00,Washington State Cabernet Sauvignon,4.00,3.00,Cabernet Sauvignon,None,None,None,None,NaN,None,23.07,"[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.5, ...",116,4.06,4.03,3.89,4.16,5608
19604,286228,6763584,1,327.00,United States,255469.00,Washington State Cabernet Sauvignon,4.00,3.00,Cabernet Sauvignon,None,None,None,None,NaN,None,23.07,"[4.0, 3.5, 4.5, 4.0, 5.0, 5.0, 4.0, 4.0, 5.0, ...",315,4.06,4.11,4.02,4.19,5608


In [32]:
df_fst = df.copy()

In [33]:
df = df_train_sample.copy()

In [34]:
def calculate_psi(expected, actual, buckettype='bins', buckets=10, axis=0):
    '''Calculate the PSI (population stability index) across all variables
    Args:
       expected: numpy matrix of original values
       actual: numpy matrix of new values, same size as expected
       buckettype: type of strategy for creating buckets, bins splits into even splits, quantiles splits into quantile buckets
       buckets: number of quantiles to use in bucketing variables
       axis: axis by which variables are defined, 0 for vertical, 1 for horizontal
    Returns:
       psi_values: ndarray of psi values for each variable
    Author:
       Matthew Burke
       github.com/mwburke
       worksofchart.com
    '''

    def psi(expected_array, actual_array, buckets):
        '''Calculate the PSI for a single variable
        Args:
           expected_array: numpy array of original values
           actual_array: numpy array of new values, same size as expected
           buckets: number of percentile ranges to bucket the values into
        Returns:
           psi_value: calculated PSI value
        '''

        def scale_range (input, min, max):
            input += -(np.min(input))
            input /= np.max(input) / (max - min)
            input += min
            return input


        breakpoints = np.arange(0, buckets + 1) / (buckets) * 100

        if buckettype == 'bins':
            breakpoints = scale_range(breakpoints, np.min(expected_array), np.max(expected_array))
        elif buckettype == 'quantiles':
            breakpoints = np.stack([np.percentile(expected_array, b) for b in breakpoints])



        expected_percents = np.histogram(expected_array, breakpoints)[0] / len(expected_array)
        actual_percents = np.histogram(actual_array, breakpoints)[0] / len(actual_array)

        def sub_psi(e_perc, a_perc):
            '''Calculate the actual PSI value from comparing the values.
               Update the actual value to a very small number if equal to zero
            '''
            if a_perc == 0:
                a_perc = 0.0001
            if e_perc == 0:
                e_perc = 0.0001

            value = (e_perc - a_perc) * np.log(e_perc / a_perc)
            return(value)

        psi_value = np.sum(sub_psi(expected_percents[i], actual_percents[i]) for i in range(0, len(expected_percents)))

        return(psi_value)

    if len(expected.shape) == 1:
        psi_values = np.empty(len(expected.shape))
    else:
        psi_values = np.empty(expected.shape[axis])

    for i in range(0, len(psi_values)):
        if len(psi_values) == 1:
            psi_values = psi(expected, actual, buckets)
        elif axis == 0:
            psi_values[i] = psi(expected[:,i], actual[:,i], buckets)
        elif axis == 1:
            psi_values[i] = psi(expected[i,:], actual[i,:], buckets)

    return(psi_values)

In [35]:
# low_wc_s100 = df[df['ratings_count_wine']<30]['ratings_average_wine'].sample(10000)
# rand_wc_s100 = df['ratings_average_wine'].sample(10000)
# calculate_psi(rand_wc_s100, low_wc_s100, buckettype='bins', buckets=10, axis=0)

In [36]:
df = df[(df.price_avg<=df.price_avg.quantile(0.99))&
        (df.price_avg>=df.price_avg.quantile(0.01))].reset_index(drop=True)

In [37]:
# european_countries = ['France', 'Italy', 'Spain', 'Portugal',
#        'Germany',  'Austria', 
#        'Greece', 'Switzerland', 'Hungary', 'Italie',
#        'Espagne', 'Croatia', 'Romania',
#        'United Kingdom', 'Slovenia', 'Allemagne', 'Netherlands', 'Frankrijk',
#        'Grèce', 'Italië', 'Autriche']

In [38]:
countries_dict = {'Espagne':'Spain','Allemagne':'Germany','Frankrijk':'France',
                  'Grèce':'Greece','Italië':'Italy','Autriche':'Austria','Italie':'Italy'}

In [39]:
df['country'] = df['country'].apply(lambda x: countries_dict[x] if x in countries_dict.keys() else x)

In [40]:
df.columns

Index(['index', 'wine_id', 'wine_type_id', 'region_id', 'country', 'winery_id',
       'style_name', 'style_body', 'acidity', 'grape_1', 'grape_2', 'grape_3',
       'grape_4', 'grape_5', 'alcohol', 'is_natural', 'price_avg', 'ratings',
       'ratings_count', 'target_rating', 'currency_rating', 'min_rating',
       'max_rating', 'seed'],
      dtype='object')

In [41]:
cols = ['wine_type_id', 'region_id',
       'country', 'winery_id',
       #'style_name', 'style_body',
       'acidity', 'grape_1', 'grape_2', 'grape_3', 'grape_4', 'grape_5', 
       'alcohol', 'is_natural', 'price_avg',
       'ratings_count', 'currency_rating', 'min_rating', 'max_rating']
target = 'target_rating'

In [42]:
cat_features=['wine_type_id', 'country', 'winery_id', 'region_id',
              #'style_name', 'style_body',
              'grape_1', 'grape_2', 'grape_3', 'grape_4', 'grape_5',
              'is_natural']
df[cat_features]=df[cat_features].astype(str)

In [43]:
df.index = df['wine_id']

In [45]:
df = df.fillna('')

In [46]:
df_model = df.copy()

In [47]:
df_model.shape

(19212, 24)

In [48]:
df_target = df_model[target].copy()
df_model = df_model[cols].copy()

In [49]:
X_train, X_test, y_train, y_test = train_test_split(df_model, df_target, test_size = 0.25, random_state = 42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25, random_state = 42)

In [50]:
model = CatBoostRegressor(iterations=500,
                             learning_rate=0.05,
                             depth=7,
                             eval_metric='RMSE',
                             random_seed = 42,
                             bagging_temperature = 0.5,
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=20)

In [51]:
model.fit(X_train, y_train,
          eval_set=(X_val, y_val),
          cat_features=cat_features,
          use_best_model=True,
          verbose=50)

0:	learn: 0.2856900	test: 0.2869452	best: 0.2869452 (0)	total: 60.4ms	remaining: 30.1s
50:	learn: 0.0662756	test: 0.0660015	best: 0.0660015 (50)	total: 470ms	remaining: 4.14s
100:	learn: 0.0566772	test: 0.0570486	best: 0.0570486 (100)	total: 886ms	remaining: 3.5s
150:	learn: 0.0552783	test: 0.0561865	best: 0.0561865 (150)	total: 1.28s	remaining: 2.97s
200:	learn: 0.0538811	test: 0.0555613	best: 0.0555601 (199)	total: 1.71s	remaining: 2.54s
250:	learn: 0.0522873	test: 0.0549277	best: 0.0549277 (250)	total: 2.13s	remaining: 2.12s
300:	learn: 0.0509341	test: 0.0544217	best: 0.0544217 (300)	total: 2.57s	remaining: 1.7s
350:	learn: 0.0498577	test: 0.0539297	best: 0.0539280 (347)	total: 3s	remaining: 1.27s
400:	learn: 0.0487823	test: 0.0535863	best: 0.0535863 (400)	total: 3.44s	remaining: 850ms
450:	learn: 0.0477733	test: 0.0532934	best: 0.0532929 (438)	total: 3.88s	remaining: 422ms
499:	learn: 0.0467309	test: 0.0529585	best: 0.0529560 (498)	total: 4.31s	remaining: 0us

bestTest = 0.05295603

In [ ]:
# quantiles = [i/10 if i!=10 else 0.99 for i in range(1,11) ]
# df['ratings_average_wine_prep'] = df['ratings_average_wine']
# for q in quantiles:
#     k = 0.1 if (q!= 0.1)and(q!= 0.99) else 0.09
#     df_bucket = df[(df.price_avg<df.price_avg.quantile(q))&
#                    (df.price_avg>df.price_avg.quantile(q-k))]
    
#     median_price = df_bucket.price_avg.median()
#     median_rating = df_bucket.ratings_average_wine.median()
#     df_bucket['ratings_average_wine'] = df_bucket['ratings_average_wine'].apply(
#         lambda x: min(x+(4.1-median_rating),5)) #4.1 - медиана верхнего бакета
#     bucket_index = df_bucket.index
#     df.loc[df_bucket.index, 'ratings_average_wine_prep'] = df_bucket.loc[df_bucket.index,'ratings_average_wine']

In [52]:
print('RMSE test:', np.sqrt(mean_squared_error(y_val, model.predict(X_val))))

print('RMSE oot:', np.sqrt(mean_squared_error(y_test, model.predict(X_test))))

print('MAE test:', (mean_absolute_error(y_val, model.predict(X_val))))

print('MAE oot:', (mean_absolute_error(y_test, model.predict(X_test))))

RMSE test: 0.052956033433127624
RMSE oot: 0.05593446567884956
MAE test: 0.03795402967288011
MAE oot: 0.03978122544797609


In [184]:
df_predict = df[df['ratings_count_wine']<df['ratings_count_wine'].quantile(0.9)]

In [185]:
df_target_1 = df_predict[target]
df_predict = df_predict[cols]

In [186]:
print('RMSE test:', np.sqrt(mean_squared_error(df_target_1, model.predict(df_predict))))
print('MAE oot:', (mean_absolute_error(df_target_1, model.predict(df_predict))))

RMSE test: 0.2448280065045329
MAE oot: 0.1834113238517226


In [217]:
df_scored = pd.DataFrame([model.predict(df_predict)]).T
df_scored.index=df_predict.index
df_scored = df_scored.rename({0:'ratings_average_wine_predicted'},axis=1)
df_scored[target] = df_target_1

In [218]:
df_scored.reset_index()[['wine_id','ratings_average_wine_predicted']].to_pickle('df_scored.pkl')